# Application: Interpolating Weak Reaction Rate Tables

[超新星闪](https://en.wikipedia.org/wiki/Supernova#Light_curves)是由于${}^{56}\mathrm{Ni}$的放射性衰变造成的，这是轻元素核聚变反应的终点之一。在高密度条件下，一旦形成${}^{56}\mathrm{Ni}$，电子俘获反应：

$${}^{56}\mathrm{Ni} + e^- \rightarrow {}^{56}\mathrm{Co} + \nu_e$$

在决定剩余${}^{56}\mathrm{Ni}$相对于其他稳定铁族核素的数量方面起着重要作用。
类似的转变也可以通过$\beta^+$衰变发生：

$${}^{56}\mathrm{Ni} \rightarrow {}^{56}\mathrm{Co} + e^+ + \nu_e$$

```{note}
我们称这些反应为*弱核反应*，因为它们涉及中子转变为质子的过程，因此与弱核力有关。
```

这些反应率依赖于温度和密度（以电子密度 $\rho Y_e$ 的形式表示，其中 $Y_e$ 是电子分数），
它们的计算值通常以表格的形式提供，
需要反应网络进行插值计算。


在这里我们将学习如何处理表格形式的反应率。


## 访问数据

这里是上述反应率的数据表：`56ni-56co_electroncapture.dat`，数据来源于 [Langanke & Martínez-Pinedo 2001](https://www.sciencedirect.com/science/article/abs/pii/S0092640X01908654)。这些数据已经经过整理，将电子俘获和 $\beta^+$ 衰变的反应率合并为单一反应率。


In [ ]:
import numpy as np

In [ ]:
!head 56ni-56co_electroncapture.dat

#56ni -> 56co, e- capture
#Q=-1.624 MeV
#
#Log(rhoY)    Log(temp)     mu              dQ    Vs     Log(e-cap-rate)        Log(nu-energy-loss)  Log(gamma-energy)
#Log(g/cm^3)  Log(K)        erg             erg   erg    Log(1/s)               Log(erg/s)           Log(erg/s)
1.000000      7.000000      -4.806530e-09  0.00  0.00   -8.684000              -1.486129e+01         -100.00
1.000000      8.000000      -9.292624e-08  0.00  0.00   -9.164000              -1.533229e+01         -100.00
1.000000      8.301030      -2.146917e-07  0.00  0.00   -9.291000              -1.544729e+01         -100.00
1.000000      8.602060      -4.902661e-07  0.00  0.00   -9.387000              -1.551729e+01         -100.00
1.000000      8.845098      -8.058948e-07  0.00  0.00   -8.777000              -1.485829e+01         -100.00


我们可以看到数据以列的形式存储，大多数量都以对数形式表示。我们关注第一列 $\log(\rho Y_e)$，第二列 $\log(T)$，以及第六列 $\log(\lambda)$（反应率）。


```{note}
这里的 $\log()$ 是以10为底的对数。
```

```{tip}
我们将使用 `np.genfromtxt()` 来读取数据，它可以将注释行解释为数据的标签，这样我们就能方便地索引各个列。
```

In [ ]:
data = np.genfromtxt("56ni-56co_electroncapture.dat", skip_header=3, names=True)

从表中可以看出，该表格包含了13个温度点和11个密度点。


In [ ]:
ntemp = 13
nrho = 11

我们可以通过每隔 `ntemp` 个值取一次的方式，从 `LogrhoY` 列中提取唯一的密度值 ($\log(\rho Y_e)$)：


In [ ]:
logrho = data["LogrhoY"][::ntemp]
logrho

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

以及从`Logtemp`列中获取的唯一温度值($\log(T)$)：


In [ ]:
logT = data["Logtemp"][0:ntemp]
logT

array([ 7.      ,  8.      ,  8.30103 ,  8.60206 ,  8.845098,  9.      ,
        9.176091,  9.30103 ,  9.477121,  9.69897 , 10.      , 10.477121,
       11.      ])

```{note}
温度点的分布并不均匀。在进行插值时，我们需要特别注意考虑这一点。
```

最后，我们将从`Logecaprate`列中获取反应率$\lambda$，并将其重塑为二维表格：


In [ ]:
lograte = data["Logecaprate"].reshape(nrho, ntemp)

以下是一些数值：


In [ ]:
lograte[0:4, 0:4]

array([[-8.684, -9.164, -9.291, -9.387],
       [-7.705, -8.165, -8.291, -8.387],
       [-6.834, -7.171, -7.293, -7.388],
       [-6.118, -6.22 , -6.31 , -6.392]])

在这种形式下，密度在第一维度（沿着行）变化，温度在第二维度（沿着列）变化。


## 双线性插值


我们将使用[双线性插值](https://en.wikipedia.org/wiki/Bilinear_interpolation)，取表中围绕我们想要计算反应率的点 $(\rho Y_e, T)$ 的4个点，并用函数拟合：

\begin{align*}
\lambda(\rho Y_e, T) = {}&a (\log(\rho Y_e) - \log(\rho Y_e)_i)(\log(T) - \log(T)_j) + \\
                       &b (\log(\rho Y_e) - \log(\rho Y_e)_i) + \\
                       &c (\log(T) - \log(T)_j) + \\
                       &d
\end{align*}

这里我们取 $(i, j)$ 作为插值的参考点。

我们需要利用表中的信息求解这4个未知数 $\{a, b, c, d\}$。


从视觉上看是这样的：

![双线性插值](bilinear_ecapture.png)

其中 $\times$ 标记了我们想要知道反应率的位置。


我们可以通过在4个点上计算插值来代数求解：

\begin{align*}
\lambda_{i,j} &= d \\
\lambda_{i+1,j} &= b \Delta (\log\rho Y_e) + d \\
\lambda_{i,j+1} &= c \Delta (\log T) + d \\
\lambda_{i+1,j+1} &= a \Delta (\log\rho Y_e) \Delta (\log T) + b \Delta (\log \rho Y_e) + c \Delta (\log T) + d
\end{align*}

之后，我们看到:

$$d = \lambda_{i,j}$$

$$b = \frac{\lambda_{i+1,j} - \lambda_{i,j}}{\Delta (\log \rho Y_e)}$$

$$c = \frac{\lambda_{i,j+1} - \lambda_{i,j}}{\Delta (\log T)}$$

$$a = \frac{\lambda_{i+1,j+1} - \lambda_{i+1,j} - \lambda_{i,j+1} + \lambda_{i,j}}{\Delta(\log \rho Y_e) \Delta( \log T)}$$

以下是该插值的实现：


In [9]:
def interp(logrhoY_v, logT_v, lograte_t, logrhoY0, logT0):
    """interpolate lograte_t to find the rate at a point (logrhoY0, logT0).
    Here logrhoY_v and logT_v are the points where the rate is tabulated"""

    i = np.argwhere(logrhoY_v > logrhoY0)[0][0] - 1
    j = np.argwhere(logT_v > logT0)[0][0] - 1

    dlogrhoY = logrhoY_v[i+1] - logrhoY_v[i]
    dlogT = logT_v[j+1] - logT_v[j]

    a = (lograte_t[i+1, j+1] - lograte_t[i+1, j] - lograte_t[i, j+1] + lograte_t[i, j]) / (dlogrhoY * dlogT)
    b = (lograte_t[i+1, j] - lograte_t[i, j]) / dlogrhoY
    c = (lograte_t[i, j+1] - lograte_t[i, j]) / dlogT
    d = lograte_t[i, j]

    return (a * (logrhoY0 - logrhoY_v[i]) * (logT0 - logT_v[j]) +
            b * (logrhoY0 - logrhoY_v[i]) +
            c * (logT0 - logT_v[j]) +
            d)

```{caution}
我们没有检查密度和温度是否在表格范围内，也没有检查它们是否超出了表格的边界限制。
```

现在我们可以试一试。


In [10]:
logrhoY0 = 3.5
logT0 = 9.4
loglambda = interp(logrho, logT, lograte, logrhoY0, logT0)
loglambda

np.float64(-4.7833819981855985)

In [11]:
print(f"at T = {10**logT0:12.6g} and (rho Y_e) = {10**logrhoY0:12.6g}, the rate is {10**loglambda:12.6g}")

at T =  2.51189e+09 and (rho Y_e) =      3162.28, the rate is  1.64671e-05


## Using SciPy

We can do the same bilinear interpolation using SciPy.

In [12]:
from scipy import interpolate

In [15]:
lrate = interpolate.interpn((logrho, logT), lograte, (3.5, 9.4))
10**lrate

array([1.64671333e-05])

We see that we get the same value (to at least 7 significant digits!)

SciPy can also do bicubic interpolation--this would require 16 points instead of the 4 points our method used.

In [16]:
lrate = interpolate.interpn((logrho, logT), lograte, (3.5, 9.4), method="cubic")
10**lrate

array([1.58899208e-05])

We see that this differs from the bilinear value in the second significant digit.